Sentiment Ananlysis of Faceebook Political Comments

In [84]:
# import libraries
from collections import defaultdict
from nltk import word_tokenize 
from nltk.util import ngrams
from nltk import RegexpTokenizer
import pandas as pd
import os

In [85]:
# bigram model for Positive Comment Corpus
positive_model = defaultdict(lambda:defaultdict(lambda: 0))

In [86]:
tokenizer = RegexpTokenizer(r"\w+")

In [87]:
# Read Positive Comments Corpus 

import csv
with open("positive_comments.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, line in enumerate(reader):
        if i != 0:
            #print(line[0])
            #token = nltk.word_tokenize(line[0].lower())
            token = tokenizer.tokenize(line[0].lower())
            bigram = list(ngrams(token, 2)) 
            for x in bigram:
                positive_model[x[0]][x[1]] += 1

In [88]:
# bigram model for Positive Comment Corpus

negative_model = defaultdict(lambda:defaultdict(lambda: 0))

In [89]:
# Read Negative Comments Corpus

with open("negative_comments.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for i, line in enumerate(reader):
        if i != 0:
            #print(line[0])
            #token = nltk.word_tokenize(line[0].lower())
            token = tokenizer.tokenize(line[0].lower())
            bigram = list(ngrams(token, 2)) 
            for x in bigram:
                negative_model[x[0]][x[1]] += 1

In [90]:
# calculating probabilities

for x in positive_model:
    total_count = float(sum(positive_model[x].values()))
    for y in positive_model[x]:
        positive_model[x][y] /= total_count
        
for x in negative_model:
    total_count = float(sum(negative_model[x].values()))
    for y in negative_model[x]:
        negative_model[x][y] /= total_count
        
        


In [91]:
# testing

def check_sentiment(comment):

    tokenized = tokenizer.tokenize(comment.lower())
    bigram_test = list(ngrams(tokenized, 2))

    positive_p = 1
    negative_p = 1

    for (x,y) in bigram_test:
        positive_p = positive_p * positive_model[x][y]
        negative_p = negative_p * negative_model[x][y]
    
    print('Probability for Postive Comment : ',positive_p)
    print('Probability for Negative Comment : ',negative_p)
    
    if (positive_p > negative_p):
        print('Sentiment: Positive')
    elif (positive_p < negative_p):
        print('Sentiment: Negative')
    else:
        print('Sentiment: Neutral')

In [94]:
# calculates the perplexity of the model using the goven comment
def perplexity():
    
    # calculate the vocabulary of the corpus
    vocabulary_count = len(positive_model) + len(negative_model)
    
    for first in positive_model:
        vocabulary_count += len(positive_model[first])
    for first in negative_model:
        vocabulary_count += len(negative_model[first])
    
    print('Vocabulary: ', vocabulary_count)
    
    bigram_prob = 1.
    for first in positive_model:
        prob_vals = positive_model[first].values()
        for prob in prob_vals:
            if prob != 0:
                bigram_prob *= prob
                
    for first in negative_model:
        prob_vals = negative_model[first].values()
        for prob in prob_vals:
            if prob != 0:
                bigram_prob *= prob
        
    print('Bigram probability: ', bigram_prob, '\n')
    
    # calculate the perplexity
    pp = (1.0 / bigram_prob) ** (1. / vocabulary_count)
    
    print('perplexity: ', pp)


In [92]:
# test

comment = 'Api rewatuna...aparade ape watina chandaya'
check_sentiment(comment)

# test end

Probability for Postive Comment :  0
Probability for Negative Comment :  0.25
Sentiment: Negative


In [93]:
# test

comment = 'Hoda adarsamath wedak .Api semada oba samagai'
check_sentiment(comment)

# test end

Probability for Postive Comment :  0.0020833333333333333
Probability for Negative Comment :  0
Sentiment: Positive


In [95]:
# perplexity

perplexity()


Vocabulary:  1353
Bigram probability:  6.561277028641357e-128 

perplexity:  1.2416544452487968
